In [1]:
! pip install cssutils

     ------------------------------------ 385.7/385.7 kB 727.7 kB/s eta 0:00:00
     ---------------------------------------- 63.0/63.0 kB 1.7 MB/s eta 0:00:00


In [4]:
from colorsys import rgb_to_hsv

# Function to convert a hex color code to an RGB tuple
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')  # Remove '#' if present
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))  # Convert hex to RGB values

# Function to calculate luminance of a color based on its RGB values
def luminance(r, g, b):
    a = [v / 255.0 for v in (r, g, b)]  # Normalize RGB values to range [0,1]
    # Apply luminance calculation formula
    a = [v / 12.92 if v <= 0.03928 else ((v + 0.055) / 1.055) ** 2.4 for v in a]
    return 0.2126 * a[0] + 0.7152 * a[1] + 0.0722 * a[2]  # Weighted sum for luminance

# Function to calculate contrast ratio between two colors
def contrast_ratio(color1, color2):
    L1 = luminance(*color1)
    L2 = luminance(*color2)
    # Contrast ratio formula
    return (L1 + 0.05) / (L2 + 0.05) if L1 > L2 else (L2 + 0.05) / (L1 + 0.05)

# Define colors in hex format
text_color_hex = '#576EAF'  # Example text color
background_color_hex = '#000000'  # Example background color

# Convert hex to RGB
text_color_rgb = hex_to_rgb(text_color_hex)
print(f"Text RGB: {text_color_rgb}")
background_color_rgb = hex_to_rgb(background_color_hex)
print(f"Background RGB: {background_color_rgb}")

# Calculate and print contrast ratio
ratio = contrast_ratio(text_color_rgb, background_color_rgb)
print(f"Contrast Ratio: {ratio:.2f}")


Text RGB: (87, 110, 175)
Background RGB: (0, 0, 0)
Contrast Ratio: 4.25


In [5]:
import requests
from bs4 import BeautifulSoup
import cssutils

# Function to extract color properties from CSS content
def extract_colors_from_css(css_content):
    parser = cssutils.CSSParser()  # Initialize CSS parser
    stylesheet = parser.parseString(css_content)  # Parse CSS content
    colors = {}  # Dictionary to store extracted colors

    for rule in stylesheet:
        if rule.type == rule.STYLE_RULE:  # Check if it's a CSS rule
            for property in rule.style:
                if property.name in ['color', 'background-color']:  # Extract color properties
                    selector = rule.selectorText  # CSS selector
                    if selector not in colors:
                        colors[selector] = {}
                    colors[selector][property.name] = property.value  # Store color values
    return colors

# Function to extract inline styles from HTML elements
def extract_colors_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')  # Parse HTML content
    colors = {}  # Dictionary to store extracted colors

    for tag in soup.find_all(style=True):  # Find elements with inline styles
        style = tag['style']
        tag_name = tag.name  # Get the tag name
        if tag_name not in colors:
            colors[tag_name] = {}

        # Extract color and background-color properties
        if 'color' in style:
            colors[tag_name]['color'] = style.split('color:')[1].split(';')[0].strip()
        if 'background-color' in style:
            colors[tag_name]['background_color'] = style.split('background-color:')[1].split(';')[0].strip()

    return colors

# Function to scrape webpage and extract colors from HTML and CSS
def scrape_colors(url):
    response = requests.get(url)  # Send request to the webpage
    html_content = response.text  # Get HTML content

    html_colors = extract_colors_from_html(html_content)  # Extract colors from HTML

    css_content = response.content.decode('utf-8')  # Decode CSS content
    css_colors = extract_colors_from_css(css_content)  # Extract colors from CSS

    return html_colors, css_colors

# Define webpage URL to scrape
url = 'https://google.com'
html_colors, css_colors = scrape_colors(url)  # Extract colors from webpage

# Print extracted colors from HTML
print("HTML Colors:")
for tag, styles in html_colors.items():
    print(f"Tag: {tag}, Styles: {styles}")

# Print extracted colors from CSS
print("\nCSS Colors:")
for selector, styles in css_colors.items():
    print(f"Selector: {selector}, Styles: {styles}")

# Organizing extracted color data
colors_data = {
    'body': {
        'text_color': html_colors.get('body', {}).get('color', ''),  # Body text color
        'background_color': html_colors.get('body', {}).get('background_color', '')  # Body background color
    },
    'h1': {
        'text_color': css_colors.get('h1', {}).get('color', '')  # H1 heading color
    },
    'mark': {
        'highlight_color': css_colors.get('.mark', {}).get('color', ''),  # Highlighted text color
        'highlight_bg': css_colors.get('.mark', {}).get('background-color', '')  # Highlighted background color
    },
    'a': {
        'link_color': css_colors.get('a', {}).get('color', ''),  # Link color
        'hover_color': css_colors.get('a:hover', {}).get('color', '')  # Hovered link color
    }
}

# Display extracted color data
for element, styles in colors_data.items():
    print(f"{element}: {styles}")


ERROR	CSSStyleRule: No start { of style declaration found: '<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en-IN"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="Zd5C0rnVUaAdXEn4VPhDEA">(function(){var _g={kEI:\'wRu7Z8uXCcSI1sQPmbjoqAM\',kEXPI:\'0,202792,3497519,1073,448528,90133,2872,2891,8349,64701,94324,266577,151788,68022,70234,11106,5230541,55,8834948,27977674,25234381,106424,10980,15164,8181,5929,5752,37749,13630,8036,6758,23878,7042,2097,740,2,2,2335,1521,168,6,154,6225,34311,29854,1346,13703,8205,3292,4134,21305,33,9041,17667,10667,10903,10441,3773,714,3853,41,13164,475,1,5539,1202,2894,1211,307,44,18880,5872,951,927,1219,4617,5773,4310,2327,1310,738,4381,1843,43,260,2,1438,3349,4960,7,2644,820,2230,2,1667,2549,9256,2801,460,459,1075,3,1120,331,15,1477,1965,714,1768,382,893,56,1026,1,212,1,615,10

HTML Colors:
Tag: a, Styles: {}
Tag: div, Styles: {}
Tag: img, Styles: {}
Tag: input, Styles: {'color': '#000'}
Tag: br, Styles: {}
Tag: p, Styles: {'color': '#70757a'}

CSS Colors:
Selector: a.gb1, a.gb4, Styles: {'color': '#00c'}
Selector: .gbi .gb4, Styles: {'color': '#dd8e27'}
Selector: .gbf .gb4, Styles: {'color': '#900'}
Selector: .h, Styles: {'color': '#1967d2'}
Selector: body, Styles: {'color': '#000'}
Selector: a, Styles: {'color': '#681da8'}
Selector: .fl a, Styles: {'color': '#1967d2'}
Selector: a:visited, Styles: {'color': '#681da8'}
Selector: .lsb, Styles: {'color': '#000'}
Selector: #SIvCob, Styles: {'color': '#474747'}
body: {'text_color': '', 'background_color': ''}
h1: {'text_color': ''}
mark: {'highlight_color': '', 'highlight_bg': ''}
a: {'link_color': '#681da8', 'hover_color': ''}
